In [4]:
#Preparing text files and converting them to json-polygon types, and also assign the values to each polygon based on their index!

import json
from collections import OrderedDict
import os

class GeoPoint:
    def __init__(self, longitude, latitude):
        self.longitude = longitude
        self.latitude = latitude


class Contour:
    def __init__(self):
        self.noPoints = 0
        self.points = []


IDWM_Contours = {}

def parse_coordinates(coordinates):
    latDeg = int(coordinates[0:2])
    latMin = int(coordinates[2:4])
    latSec = int(coordinates[4:6])
    NS = coordinates[6]
    lonDeg = int(coordinates[7:10])
    lonMin = int(coordinates[10:12])
    lonSec = int(coordinates[12:14])
    EW = coordinates[14]

    longitude = lonDeg + lonMin / 60.0 + lonSec / 3600.0

    if EW == "W":
        longitude = -longitude    

    latitude = latDeg + latMin / 60.0 + latSec / 3600.0

    if NS == "S":
        latitude = -latitude

    return longitude, latitude

def parse_lines(lines):
    for line in lines:
        trimmed = line.strip()
        parts = trimmed.split()

        if len(parts) == 4:  # index reference 10001 1 5745 0
            contourID = int(parts[0])
            first = int(parts[1])
            last = int(parts[2])
            dummy = int(parts[3])

            if contourID not in IDWM_Contours:
                IDWM_Contours[contourID] = Contour()

            IDWM_Contours[contourID].noPoints = last
        elif len(parts) == 2:  # points
            coordinates, point_no = parts[0], int(parts[1])

            if contourID not in IDWM_Contours:
                IDWM_Contours[contourID] = Contour()

            longitude, latitude = parse_coordinates(coordinates)
            IDWM_Contours[contourID].points.append(GeoPoint(longitude, latitude))


def geo_point_to_dict(geo_point):
    return OrderedDict([("type", "Point"), ("coordinates", [geo_point.longitude, geo_point.latitude])])


def contour_to_json(contour, value):
    geometry = OrderedDict([
        ("type", "Polygon"),
        ("coordinates", [[(point.longitude, point.latitude) for point in contour.points]])
    ])
    feature = OrderedDict([
        ("type", "Feature"),
        ("geometry", geometry),
        ("contour_id", contour_id),
        ("value", value)
    ])
    
    return json.dumps(feature, indent=2)


def read_values_file(file_path):
    values = {}
    with open(file_path, "r") as file:
        for line in file:
            parts = line.strip().split("\t")
            contour_id = parts [0]
            value = " ".join(parts[1:3])
            values[int(contour_id)] = value
            #print(f"Contour ID: {contour_id}, Value: {value}")
    return values
    


file_path = "D:/1_______internship-itu/IDWM-flat files/REG2/VOLMET/LINES.txt"


with open(file_path, "r") as file:
    lines = file.readlines()

parse_lines(lines)

values_file_path = "D:/1_______internship-itu/IDWM-flat files/REG2/VOLMET/VOLMET_LINESIDX.DAT"
contour_values = read_values_file(values_file_path)



output_directory="D:/1_______internship-itu/IDWM-flat files/REG2/VOLMET/OUTPUT/JSONs"
for contour_id, contour in IDWM_Contours.items():
    json_data = contour_to_json(contour, contour_values.get(contour_id, 0))  # Assign a default value of 0 if not found in the file
    output_file_path = os.path.join(output_directory,f"contour_{contour_id}.json")

    with open(output_file_path, "w") as output_file:
        output_file.write(json_data)

        
        
print("done")  

ValueError: invalid literal for int() with base 10: '10402   262 V-AFI-A'

In [10]:
###Additional lines for correcting the contours with the IDL problem
###split the contours by this code!
import os
import json
from collections import OrderedDict

class GeoPoint:
    def __init__(self, longitude, latitude):
        self.longitude = longitude
        self.latitude = latitude


class Contour:
    def __init__(self):
        self.noPoints = 0
        self.points = []

def separate_polygon(points):
    l1 = []
    l2 = []

    for gp in points:
        if gp.longitude < -47:
            l1.append(gp)
        else:
            l2.append(gp)
            
    #print("Avg Longitude:", avg_longitude)
    print("Points in L1:", len(l1))
    print("Points in L2:", len(l2))

    return l1, l2

def close_contour_if_needed(points):

    if len (points) >= 2 and points[0] != points[-1]:
        points.append(points[0])

IDWM_Contours = {}

def parse_coordinates(coordinates, contourID):
    latDeg = int(coordinates[0:2])
    latMin = int(coordinates[2:4])
    latSec = int(coordinates[4:6])
    NS = coordinates[6]
    lonDeg = int(coordinates[7:10])
    lonMin = int(coordinates[10:12])
    lonSec = int(coordinates[12:14])
    EW = coordinates[14]

    longitude = lonDeg + lonMin / 60.0 + lonSec / 3600.0

    if EW == "W":
        longitude= -longitude  

    latitude = latDeg + latMin / 60.0 + latSec / 3600.0

    if NS == "S":
        latitude = -latitude

    return longitude, latitude
    
def parse_lines(lines):
    contourID = None  # Initialize contourID outside the loop
    for line in lines:
        trimmed = line.strip()
        parts = trimmed.split()
        
        if len(parts) == 4:
            contourID = int(parts[0])
            if contourID not in IDWM_Contours:
                IDWM_Contours[contourID] = Contour()
            IDWM_Contours[contourID].noPoints = int(parts[2])
        elif len(parts) == 2 and contourID is not None:
            coordinates, point_no = parts[0], int(parts[1])
            longitude, latitude = parse_coordinates(coordinates, contourID)
            IDWM_Contours[contourID].points.append(GeoPoint(longitude, latitude))          

def geo_point_to_dict(geo_point):
    return OrderedDict([("type", "Point"), ("coordinates", [geo_point.longitude, geo_point.latitude])])


def contour_to_json(contour, value):
    geometry = OrderedDict([
        ("type", "Polygon"),
        ("coordinates", [[(point.longitude, point.latitude) for point in contour.points]])
    ])
    feature = OrderedDict([
        ("type", "Feature"),
        ("geometry", geometry),
        (("value", value)),
        ("contour",int(contour_id))
    ])
    return json.dumps(feature, indent=2)

#when we have two value columns
def read_values_file(file_path):
    values = {}
    with open(file_path, "r") as file:
        for line in file:
            parts = line.strip().split("\t")
            contour_id = parts [0]
            value = " ".join(parts[1:3])
            values[int(contour_id)] = value
            #print(f"Contour ID: {contour_id}, Value: {value}")
    return values

#when we have only 1 value
#def read_calues_file(file_path):
    #values = {}
    #with open(file_path, "r") as file:
    #    for line in file:
     #       contour_id, value = line.strip().split("\t")
      #      values[int(contour_id)] = float(value)
    #return values

        
file_path = "D:/1_______internship-itu/IDWM-flat files/REG2/VOLMET/LINES.txt"


with open(file_path, "r") as file:
    lines = file.readlines()

parse_lines(lines)

values_file_path = "D:/1_______internship-itu/IDWM-flat files/REG2/VOLMET/VOLMET_LINESIDX-edited.txt"
contour_values = read_values_file(values_file_path)

for contour_id in [10413,10412,10411,10410]:
    if contour_id in IDWM_Contours:
        l1, l2 = separate_polygon(IDWM_Contours[contour_id].points)
        IDWM_Contours[contour_id].points = l1
        if len(l2) > 0:
            new_contour_id = contour_id + 1000
            new_contour = Contour()  # Create a new contour instance
            new_contour.points = l2
            IDWM_Contours[new_contour_id] = new_contour  # Add the new contour to the dictionary
            #print("New Contour ID:", new_contour_id)
            #print("Number of Points in New Contour:", len(new_contour.points))
            
        
        close_contour_if_needed(IDWM_Contours[contour_id].points)
        #print("Contour", contour_id, "Points after closure:", [gp.longitude for gp in IDWM_Contours[contour_id].points])


        
        
output_directory="D:/1_______internship-itu/IDWM-flat files/REG2/VOLMET/OUTPUT/new"
for contour_id, contour in IDWM_Contours.items():
    json_data = contour_to_json(contour, contour_values.get(contour_id, 0))  # Assign a default value of 0 if not found in the file
    output_file_path = os.path.join(output_directory,f"contour_{contour_id}.json")

    with open(output_file_path, "w") as output_file:
        output_file.write(json_data)
        
       
# Print the points of the updated contours
#for contour_id in [10004, 10005, 10006]:
    #if contour_id in IDWM_Contours:
        #print("Contour", contour_id, "Points:", IDWM_Contours[contour_id].points)

# Add this line at the beginning of the script to see if it's reaching this point
#print("Starting the script...")

# Add this line after parsing the lines to check the contents of IDWM_Contours
#print(IDWM_Contours)

# Add this line after reading the values file to check the contour_values dictionary
#print(contour_values)

# Add this line inside the loop to check the generated JSON data
#print(json_data)
print("done")

Points in L1: 243
Points in L2: 291
Points in L1: 207
Points in L2: 200
Points in L1: 110
Points in L2: 281
Points in L1: 24
Points in L2: 209
done


In [12]:
#Convert the json files to one gpkg file

import os
import json
import pandas as pd
import geopandas as gpd
from shapely.geometry import shape

# Create an empty DataFrame to hold the data
data = []

# Set the path to the directory containing the JSON files
json_files_directory = "D:/1_______internship-itu/IDWM-flat files/REG2/RDARA/OUTPUT/new"

# Loop through the JSON files and read them into the DataFrame
for file_name in os.listdir(json_files_directory):
    if file_name.endswith(".json"):
        file_path = os.path.join(json_files_directory, file_name)
        with open(file_path, "r") as file:
            json_data = json.load(file)
            data.append(json_data)

# Convert the list of JSON data to a DataFrame
df = pd.DataFrame(data)

# Convert the DataFrame to a GeoDataFrame by creating a 'geometry' column
geometry = [shape(geom) for geom in df['geometry']]
gdf = gpd.GeoDataFrame(df, geometry=geometry)

# Convert the GeoDataFrame to a GeoPackage file
output_directory = "D:/1_______internship-itu/IDWM-flat files/REG2/RDARA/OUTPUT/new"
output_gpkg_file = os.path.join(output_directory, "output_file.gpkg")
gdf.to_file(output_gpkg_file, driver="GPKG")

print("done")



#output_gpkg_file = "output_file.gpkg"
#gdf.to_file(output_gpkg_file, driver="GPKG")


C:\Users\Fateme\anaconda3\lib\site-packages\geopandas\io\file.py:299: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


done


In [63]:
#Convert the json files to one json file

import os
import json

# Create an empty list to hold the JSON data
json_data_list = []

# Set the path to the directory containing the JSON files
json_files_directory = "C:/Users/Pooya System/Documents/Python Scripts/REG2/CST/CLO134/CLO134_OUTPUT/CLO50-H"

# Loop through the JSON files and read them into the list
for file_name in os.listdir(json_files_directory):
    if file_name.endswith(".json"):
        file_path = os.path.join(json_files_directory, file_name)
        with open(file_path, "r") as file:
            json_data = json.load(file)
            json_data_list.append(json_data)

# Save the list of JSON data as a single JSON file with an array of objects
output_json_file = "output_file.json"
with open(output_json_file, "w") as file:
    json.dump(json_data_list, file, indent=2)

In [64]:
#To convert the json files to one Geojson, in order to be readable in QGIS

import os
import json
import pandas as pd
import geopandas as gpd
from shapely.geometry import shape

# Create an empty DataFrame to hold the data
data = []

# Set the path to the directory containing the JSON files
json_files_directory = "C:/Users/Pooya System/Documents/Python Scripts/REG2/CST/CLO134/CLO134_OUTPUT/CLO50-H"

# Loop through the JSON files and read them into the DataFrame
for file_name in os.listdir(json_files_directory):
    if file_name.endswith(".json"):
        file_path = os.path.join(json_files_directory, file_name)
        with open(file_path, "r") as file:
            json_data = json.load(file)
            data.append(json_data)

# Convert the list of JSON data to a DataFrame
df = pd.DataFrame(data)

# Convert the DataFrame to a GeoDataFrame by creating a 'geometry' column
geometry = [shape(geom) for geom in df['geometry']]
gdf = gpd.GeoDataFrame(df, geometry=geometry)

# Convert the GeoDataFrame to a GeoJSON file
output_geojson_file = "output_file.geojson"
gdf.to_file(output_geojson_file, driver="GeoJSON")
